In [65]:
import _threshold_utils as threshold_utils
import altair as alt

import pandas as pd
import geopandas as gpd
from segment_speed_utils.project_vars import analysis_date
from shared_utils import calitp_color_palette as cp

In [66]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [67]:
GCS_PATH = "gs://calitp-analytics-data/data-analyses/rt_segment_speeds/"

In [68]:
analysis_date

'2023-03-15'

In [69]:
speed_stops_subset = ['gtfs_dataset_key', '_gtfs_dataset_name', 'shape_array_key',
       'stop_sequence', 'trip_id','speed_mph']

In [70]:
speed_stops = pd.read_parquet(f"{GCS_PATH}speeds_stop_segments_{analysis_date}")

In [71]:
speed_stops.sample()

,gtfs_dataset_key,_gtfs_dataset_name,shape_array_key,stop_sequence,trip_id,min_time,min_dist,max_time,max_dist,meters_elapsed,sec_elapsed,speed_mph
__null_dask_index__,,,,,,,,,,,,
785877,65d9589130415c685b89f4f7c2d8bd7e,LA Metro Bus Vehicle Positions,a046abda31fc54621680d3b8581b6653,37,10106000631946-DEC22,2023-03-15 20:23:46,18.85,2023-03-15 20:24:46,708.11,689.26,60.00,25.70


In [72]:
speed_stops2 = speed_stops[speed_stops_subset]

In [73]:
speed_stops2.sample()

,gtfs_dataset_key,_gtfs_dataset_name,shape_array_key,stop_sequence,trip_id,speed_mph
__null_dask_index__,,,,,,
508504,609173a0aa18c6e168c54b8731584ffa,Bay Area 511 Tri-Valley Wheels VehiclePositions,8542e0582e08cc4d26484bacd20a09dc,25,762762,14.16


In [74]:
# Do I need geometry? Doesn't seem like it..
avg_speeds = gpd.read_parquet(f"{GCS_PATH}avg_speeds_stop_segments_{analysis_date}.parquet")

In [75]:
avg_speeds_subset = ['geometry','geometry_arrowized','district','district_name']

In [76]:
avg_speeds2 = avg_speeds.drop(columns = avg_speeds_subset)

In [77]:
avg_speeds2.head(2)

,shape_array_key,stop_sequence,gtfs_dataset_key,loop_or_inlining,avg_speed_mph,n_trips,p20_speed_mph,p80_speed_mph
0,00093e1c28352239174c92c4f07a483b,1,03cadbffef6f3cd4ce839b218bc65e82,0,0.00,26,0.00,0.00
1,00093e1c28352239174c92c4f07a483b,2,03cadbffef6f3cd4ce839b218bc65e82,0,0.63,26,0.22,0.69


In [78]:
m1 = pd.merge(avg_speeds2, speed_stops2, on = ['gtfs_dataset_key','shape_array_key', 'stop_sequence'], how = 'inner')

In [79]:
avg_speeds2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155454 entries, 0 to 144863
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   shape_array_key   155454 non-null  object 
 1   stop_sequence     155454 non-null  int16  
 2   gtfs_dataset_key  155454 non-null  object 
 3   loop_or_inlining  155454 non-null  int64  
 4   avg_speed_mph     155454 non-null  float64
 5   n_trips           155454 non-null  int64  
 6   p20_speed_mph     155454 non-null  float64
 7   p80_speed_mph     155454 non-null  float64
dtypes: float64(3), int16(1), int64(2), object(2)
memory usage: 9.8+ MB


In [80]:
m1.head()

,shape_array_key,stop_sequence,gtfs_dataset_key,loop_or_inlining,avg_speed_mph,n_trips,p20_speed_mph,p80_speed_mph,_gtfs_dataset_name,trip_id,speed_mph
0,00093e1c28352239174c92c4f07a483b,1,03cadbffef6f3cd4ce839b218bc65e82,0,0.00,26,0.00,0.00,Bay Area 511 Santa Clara Transit VehiclePositions,3288092,0.00
1,00093e1c28352239174c92c4f07a483b,1,03cadbffef6f3cd4ce839b218bc65e82,0,0.00,26,0.00,0.00,Bay Area 511 Santa Clara Transit VehiclePositions,3288102,0.00
2,00093e1c28352239174c92c4f07a483b,1,03cadbffef6f3cd4ce839b218bc65e82,0,0.00,26,0.00,0.00,Bay Area 511 Santa Clara Transit VehiclePositions,3288115,0.00
3,00093e1c28352239174c92c4f07a483b,1,03cadbffef6f3cd4ce839b218bc65e82,0,0.00,26,0.00,0.00,Bay Area 511 Santa Clara Transit VehiclePositions,3288117,0.00
4,00093e1c28352239174c92c4f07a483b,1,03cadbffef6f3cd4ce839b218bc65e82,0,0.00,26,0.00,0.00,Bay Area 511 Santa Clara Transit VehiclePositions,3288099,0.00


In [81]:
m1.columns

Index(['shape_array_key', 'stop_sequence', 'gtfs_dataset_key',
       'loop_or_inlining', 'avg_speed_mph', 'n_trips', 'p20_speed_mph',
       'p80_speed_mph', '_gtfs_dataset_name', 'trip_id', 'speed_mph'],
      dtype='object')

In [82]:
m1.groupby(['_gtfs_dataset_name','shape_array_key']).agg({'n_trips':'max'}).head()

n_trips
_gtfs_dataset_name              shape_array_key                          
Anaheim Resort VehiclePositions 11d91cab41cde51a6d4f623b9cba867c        4
                                1e48661d195c13fc91d1a2f96f1f8c4f       26
                                29d2bbdbeaec1d6888800f85bebf6e33        8
                                2d6ca8831b70c1ff44dc5aa81d9114d0        5
                                39fc40d2ccadfa46548cbf953879ff90        4

In [83]:
test1 = m1.melt(id_vars=[ '_gtfs_dataset_name','shape_array_key','trip_id', 'stop_sequence','gtfs_dataset_key','loop_or_inlining',
                       'n_trips'], value_vars=[ 'avg_speed_mph','speed_mph','p20_speed_mph', 'p80_speed_mph'])

In [84]:
test1.shape

(11174816, 9)

In [85]:
# test1[test1.shape_array_key == "29d2bbdbeaec1d6888800f85bebf6e33"]

In [86]:
# Only need average speed/p20 speed/p80 to show up once for each stop sequence-operator-shape array
test2 = test1.drop_duplicates(subset = [ '_gtfs_dataset_name','shape_array_key','stop_sequence','gtfs_dataset_key','variable','value']).reset_index(drop = True)

In [87]:
test2.shape

(2842750, 9)

In [88]:
test2.shape_array_key.unique()

array(['00093e1c28352239174c92c4f07a483b',
       '001254fc8105d01a8064046249c0ceba',
       '00b40413c13a48046de6e2338aee0410', ...,
       '754c5b012195800c38dc58e72e4f482e',
       'e3c5ed2c6fa6cd5c5cd57d46aeb3cd8e',
       'efa0f969b4499620b80c9b82170e2e60'], dtype=object)

### Test with a few routes first
* Create new col that rounds up speed for plotting purposes only.

In [89]:
subset = test2[test2.shape_array_key.isin(["29d2bbdbeaec1d6888800f85bebf6e33",'754c5b012195800c38dc58e72e4f482e',
       'e3c5ed2c6fa6cd5c5cd57d46aeb3cd8e'])]

In [90]:
subset = threshold_utils.pre_clean(subset)

In [91]:
subset['Route'] = subset['Gtfs Dataset Name'] + ' ' + subset['Shape Array Key']

In [92]:
subset = subset.rename(columns = {'Value':'Speed'})

In [93]:
subset.columns

Index(['Gtfs Dataset Name', 'Shape Array Key', 'Trip Id', 'Stop Sequence',
       'Gtfs Dataset Key', 'Loop Or Inlining', 'N Trips', 'Variable', 'Speed',
       'Route'],
      dtype='object')

In [94]:
subset['Rounded Speed'] = subset.Speed.fillna(0).astype(int)

In [95]:
subset.Variable = subset.Variable.str.title().str.replace("_"," ")

In [96]:
subset.columns

Index(['Gtfs Dataset Name', 'Shape Array Key', 'Trip Id', 'Stop Sequence',
       'Gtfs Dataset Key', 'Loop Or Inlining', 'N Trips', 'Variable', 'Speed',
       'Route', 'Rounded Speed'],
      dtype='object')

In [97]:
# One df for the actual speeds
subset_speedmph = subset[subset.Variable == 'Speed Mph'].reset_index(drop = True)

In [98]:
# One df for the percentiles
subset_other= subset[subset.Variable != 'Speed Mph'].reset_index(drop = True)

In [99]:
dropdown_list = subset_speedmph['Route'].unique().tolist()

In [100]:
dropdown_list

['Anaheim Resort VehiclePositions 29d2bbdbeaec1d6888800f85bebf6e33',
 'Eastern Sierra Vehicle Positions 754c5b012195800c38dc58e72e4f482e',
 'Eastern Sierra Vehicle Positions e3c5ed2c6fa6cd5c5cd57d46aeb3cd8e']

In [101]:
test = (alt.Chart(subset_other, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("Rounded Speed:Q", axis=alt.Axis(labelAngle=360)),
            color=alt.Color(
                "Variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=subset_other.columns.tolist(),
            column=alt.Column(
                "Stop Sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title="mytitle"))

In [104]:
def create_dot_plot1(df, col_for_dots: str, 
                    x_axis_col:str, y_axis_col:str,
                   tooltip_cols:list, chart_title:str):
  
    chart = (alt.Chart(df).mark_circle(opacity=1, size = 400).transform_window(
    id='rank()',
    groupby=[col_for_dots]).encode(
    alt.X(f'{x_axis_col}:O', sort='descending', 
          axis=alt.Axis(ticks=False, grid=True)),
    alt.Y(f'{y_axis_col}:N'), 
    color=alt.Color(f"{col_for_dots}:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS), legend=None),
    tooltip = tooltip_cols)
             .properties(title = chart_title))
    
    return chart

In [105]:
test2 = create_dot_plot1(subset_speedmph, 'Variable', 'Stop Sequence', 'Rounded Speed', subset_speedmph.columns.tolist(),  'Percentile/Average')

In [107]:
test2

alt.Chart(...)

In [108]:
def create_jitter_plot(df):
    
    title_op = df['Gtfs Dataset Name'].iloc[0].replace('VehiclePositions','').strip()
    inline = df['Loop Or Inlining'].iloc[0]
    
    dropdown_list = df['Route'].unique().tolist()
    # Init first op doesn't work after I downloaded altair 5...try again alter
    input_dropdown = alt.binding_select(options=sorted(dropdown_list), name="Operator/Shape Array Key")
    selection = alt.selection_single(name="Operator/Shape Array Key",
    fields=['Route'],
    bind=input_dropdown)

    chart1 = (
        alt.Chart(df, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("Rounded Speed:Q", axis=alt.Axis(labelAngle=360)),
            color=alt.Color(
                "Variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=df.columns.tolist(),
            column=alt.Column(
                "Stop Sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title=f"{title_op} - Route Type {inline}")
    )
    
    chart1 = threshold_utils.chart_size(chart1, 40, 250)
    
    return chart1

In [109]:
chart1 = create_jitter_plot(subset_speedmph)

/opt/conda/lib/python3.9/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'


In [110]:
chart2 = create_jitter_plot(subset_other)

In [111]:
chart1 | chart2

alt.HConcatChart(...)

In [115]:
(
        alt.Chart(subset_other, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("Rounded Speed:Q", axis=alt.Axis(labelAngle=360, ticks=False, grid=False, labels=False)),
            color=alt.Color(
                "Variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=subset_other.columns.tolist(),
            column=alt.Column(
                "Stop Sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title="my title")
    )
    

alt.Chart(...)

### Draft

In [ ]:
def alt_dropdown(df, col_for_dropdown:str, dropdown_menu_title:str):
    # Create dropdown menu
    # Exclude "none" operators which are only scheduled data
    df = df.loc[df[col_for_dropdown] != "None"][[col_for_dropdown]]
    dropdown_list = df[col_for_dropdown].unique().tolist()
    
    # Show only first operator by default
    initialize_first_op = sorted(dropdown_list)[0]
    input_dropdown = alt.binding_select(options=sorted(dropdown_list), name=dropdown_menu_title)
    
    selection = alt.selection_single(name= dropdown_menu_title,fields=[col_for_dropdown],
    bind=input_dropdown, init={col_for_dropdown: initialize_first_op})
                 
    return selection

In [ ]:
def create_jitter_plot(df):
    
    title_op = df['Gtfs Dataset Name'].iloc[0].replace('VehiclePositions','').strip()
    inline = df['Loop Or Inlining'].iloc[0]
    
    chart1 = (
        alt.Chart(df, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("Rounded Speed:Q", axis=alt.Axis(labelAngle=360)),
            color=alt.Color(
                "Variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=df.columns.tolist(),
            column=alt.Column(
                "Stop Sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title=f"{title_op} - Route Type {inline}")
    )
    
    chart1 = threshold_utils.chart_size(chart1, 40, 250)
    
    return chart1

In [ ]:
chart2 = (
        alt.Chart(anaheim_test, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .configure_facet(spacing=0)
        .configure_view(stroke=None)
        .properties(title="Trip Duration by RT Category")
    )

In [ ]:
chart2 = threshold_utils.chart_size(chart2,80,300)

In [ ]:
chart2

In [ ]:
chart1 = (
        alt.Chart(anaheim_test_speedmph, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "stop_sequence:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .configure_facet(spacing=0)
        .configure_view(stroke=None)
        .properties(title=f"Loop/Inling:{anaheim_test_speedmph.loop_or_inlining.iloc[0]}")
    )

In [ ]:
chart1 = threshold_utils.chart_size(chart1,80,300)

In [ ]:
chart1

In [ ]:
def create_dot_plot2(df, col_for_dots: str, 
                    x_axis_col:str, y_axis_col:str,
                   tooltip_cols:list, chart_title:str):
  
    chart = (alt.Chart(df).mark_circle(opacity=1, size = 100).transform_window(
    id='rank()',
    groupby=[col_for_dots]).encode(
    alt.X(f'{x_axis_col}:O', sort='descending', 
          axis=alt.Axis(ticks=False, grid=True)),
    alt.Y(f'{y_axis_col}:N'), 
    color=alt.Color(f"{col_for_dots}:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS), legend=None),
    tooltip = tooltip_cols)
             .properties(title = chart_title))
    
    return chart

In [ ]:
chart3 = create_dot_plot1(anaheim_test_other, 'variable', 'stop_sequence', 'rounded_speed', anaheim_test_other.columns.tolist(),  'Percentile/Average')

In [ ]:
chart3 = threshold_utils.chart_size(chart3,650,300)

In [ ]:
chart4 = create_dot_plot2(anaheim_test_speedmph, 'variable', 'stop_sequence', 'rounded_speed', anaheim_test_speedmph.columns.tolist(), 'Speed per Trip')

In [ ]:
chart4 = threshold_utils.chart_size(chart4,650,300)

In [ ]:
chart4

In [ ]:
chart3 + chart4

In [ ]:
chart7 = (
        alt.Chart(anaheim_test_other, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=-90,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .configure_facet(spacing=0)
        .configure_view(stroke=None)
        .properties(title="Trip Duration by RT Category")
    )

In [ ]:
chart7 = threshold_utils.chart_size(chart7,80,300)

In [ ]:
chart8 = (
        alt.Chart(anaheim_test_other, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=-90,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title="Trip Duration by RT Category")
    )

In [ ]:
chart9 = (
        alt.Chart(anaheim_test_speedmph, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title=f"Loop/Inling:{anaheim_test_speedmph.loop_or_inlining.iloc[0]}")
    )

In [ ]:
chart8

In [ ]:
chart9 | chart8

In [ ]:
# pip install altair==5.0.0rc3
chart5 = alt.Chart(anaheim_test_speedmph, title='Normally distributed jitter').mark_circle(size=50).encode(
    y="rounded_speed:Q",
    x="stop_sequence:N",
    yOffset="jitter:Q",
    color=alt.Color('stop_sequence:Q').legend(None)
).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
)


In [ ]:
chart5 = threshold_utils.chart_size(chart5,650,300)

In [ ]:
chart5